# Introduction

Welcome to chordparser's Colab notebook! chordparser serves to provide a framework for analysing chords. Chord notation can be parsed into Chords, which can then be analysed against other chords or the key of the song. This allows for harmonic analysis in chord sheets and helps musicians understand why and how chord progressions work.

This notebook is written for two audiences:

1. Non-programmers interested in what chordparser can do and who want to use it without any programming
2. Programmers interested in getting started on using chordparser for their projects

As such, the notebook serves as a **Showcase** and a **Tutorial**. It will show you what chordparser can do with a working example. 

People in Category 1 might want to ignore textboxes labelled "*Code*" and focus on the _code output_ which show what you can use chordparser for. You might also wish to refer to the Setup section (look for _**Try it out for yourself!**_) on how you can use this notebook for your own chord sheets with little hassle.

Those in Category 2 might want to follow along the "*Code*"-labelled textboxes, which describe what we are doing in the _code cells_. By the end of it, you would have written a script that can parse and analyse a chord sheet.

Without further ado, let's go to the working example!

_Check out the [Github page](https://github.com/titus-ong/chordparser) and the [full documentation](https://chordparser.readthedocs.io/en/latest/) for more information._

# Working example

In this example, we will be using chordparser to parse the chords in a song, transpose the chords, convert them to Roman numeral notation, and analyse the song's chord progression.

## Setup

*Code*: Before we start, we'll have to import the module and initialise the Parser. The Parser is the central factory of chordparser where you can create, manipulate and analyse musical objects. 

In [ ]:
!pip install chordparser
import chordparser
cp = chordparser.Parser()

A sample chord sheet is provided in chordparser as ``Parser.sample`` (All I Want For Christmas Is You by Mariah Carey). Let's start by grabbing it from the package:

_**Try it out for yourself!** Just follow these instructions:_

1. *Save a copy of this notebook in your Drive (File -> save a copy in Drive)*
2. *In the code cell below, uncomment the commented lines (remove the #'s from lines 2-6) and comment the code lines below them (put #'s in front of lines 7-8)*
3. *In the following code cell, change the inputs "G", "major" to the key of your song (e.g. "D", "minor")*
4. *Press Ctrl+F9 to run all cells and click on the "Choose File" button in the code cell below to upload your own file*

*Note: the results will only display the first ten lines of the sheet. Remove '[:10]' in the code cells where you want to view the entire chord sheet.*

In [ ]:
import pprint
# from google.colab import files
# upload = files.upload()
# val = list(upload.values())
# sheet = val[0].decode('utf-8').replace('\r', '')
# print(sheet)
sheet = cp.sample
print(sheet[:420])  # show the first ten lines

{t:All I Want For Christmas Is You}
{key:G}
{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}

[G] I don't want a lot for Christmas there is just one thing I [G7] need
[C] I don't care about the presents [Cm] underneath the Christmas tree
[G] I just want you for my [Gaug] own
[Em7] more than you could ever [Cm] know
[G] Make my wish come [E7] true
[Amadd9] All I want for [D7] Christmas is [G] you [Em7] [C] [D7]


You can either specify the song's key (in this case, G major) or find it using regex. For simplicity's sake, let's specify the key and create the scale:

In [ ]:
# change the inputs to your song's key if you are using your own chord sheet
key = cp.create_key("G", "major")
scale = cp.create_scale(key)
scale

G major scale

Next, let's separate the chord notations from the lyrics.

*Code*: We'll store the notations and lyrics in separate lists so we can join them back anytime we want. It'll be nice to keep a similar format to the sheet, so we'll extract the notations and lyrics line-by-line using regex and store them in nested lists:

In [ ]:
import re

# we'll define a function so we can re-use it later
def get_notations_lyrics(sheet):
    sheet_list = sheet.split('\n')
    notations_nlist = []
    lyrics_nlist = []
    for line in sheet_list:
        notations = re.findall(r'(?:\[(.+?)\])', line)
        notations_nlist.append(notations)
        lyrics = re.split(r'\[.+?\]', line)
        lyrics_nlist.append(lyrics)
    return notations_nlist, lyrics_nlist

notations_nlist, lyrics_nlist = get_notations_lyrics(sheet)
print("Chords:")
pprint.pprint(notations_nlist[:10])
print("\nLyrics:")
pprint.pprint(lyrics_nlist[:10])

Chords:
[[],
 [],
 [],
 [],
 ['G', 'G7'],
 ['C', 'Cm'],
 ['G', 'Gaug'],
 ['Em7', 'Cm'],
 ['G', 'E7'],
 ['Amadd9', 'D7', 'G', 'Em7', 'C', 'D7']]

Lyrics:
[['{t:All I Want For Christmas Is You}'],
 ['{key:G}'],
 ['{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}'],
 [''],
 ['', " I don't want a lot for Christmas there is just one thing I ", ' need'],
 ['', " I don't care about the presents ", ' underneath the Christmas tree'],
 ['', ' I just want you for my ', ' own'],
 ['', ' more than you could ever ', ' know'],
 ['', ' Make my wish come ', ' true'],
 ['', ' All I want for ', ' Christmas is ', ' you ', ' ', ' ', '']]


Finally, we have to convert these chord notations into Chords for chordparser to use them.

*Code*: Chords are musical objects that have tons of useful functions which we'll see later. With the help of ``Parser.create_chord(notation)``, it's a simple matter of iterating over the list of notations and plugging them into the Parser to get Chords:

In [ ]:
def get_chords_lyrics(sheet):
    notations_nlist, lyrics_nlist = get_notations_lyrics(sheet)
    chords_nlist = []
    for line in notations_nlist:
        chords_nlist.append([])
        for notation in line:
            chords_nlist[-1].append(cp.create_chord(notation))
    return chords_nlist, lyrics_nlist

chords_nlist, lyrics_nlist = get_chords_lyrics(sheet)
pprint.pprint(chords_nlist[:10])

[[],
 [],
 [],
 [],
 [G chord, G7 chord],
 [C chord, Cm chord],
 [G chord, Gaug chord],
 [Em7 chord, Cm chord],
 [G chord, E7 chord],
 [Amadd9 chord, D7 chord, G chord, Em7 chord, C chord, D7 chord]]


To get the original sheet back, we have to merge the notations and lyrics together.

*Code*: We are going to merge the two nested lists together line-by-line. We've gotten the new Chords list which can display proper notation when printed, so let's try merging it with the lyrics nested list:

In [ ]:
def merge(lyrics, chords) -> str:
    merged = ""
    for i in range(len(lyrics)):
        line = "".join(
            x + "[" + str(y) + "]"  # str(Chord) for nice printing of Chords
            for x, y in zip(lyrics[i], chords[i])
        )
        # zip iterates over smallest list, and lyrics has one more element than
        # chords per line
        merged += line + lyrics[i][-1] + "\n"
    return merged[:-1]

print(merge(lyrics_nlist[:10], chords_nlist[:10]))  # show first ten lines

{t:All I Want For Christmas Is You}
{key:G}
{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}

[G] I don't want a lot for Christmas there is just one thing I [G7] need
[C] I don't care about the presents [Cm] underneath the Christmas tree
[G] I just want you for my [Gaug] own
[Em7] more than you could ever [Cm] know
[G] Make my wish come [E7] true
[Amadd9] All I want for [D7] Christmas is [G] you [Em7] [C] [D7]


That's it! With these lists and functions in hand, we are now ready to rock. 

## Transposing

Let's start off with transposing the song into a different key. It's currently in the key of G major, so let's try transposing it to C♯ major. 

*Code*: To do so, we're going to use ``Parser.get_tone_letter(*notes)`` to get the correct semitone and letter intervals between G and C♯ so that the transposition is accurate. Then, with the help of ``Chord.transpose(semitones, letters)``, we can easily transpose the Chords correctly by iterating over the Chords nested list:

In [ ]:
new_key = cp.create_key("C#")
((semitones, letters),) = cp.get_tone_letter(key.root, new_key.root)
# semitones = 6, letters = 3

chords_nlist, lyrics_nlist = get_chords_lyrics(sheet)

for line in chords_nlist:
    for chord in line:
        chord.transpose(semitones, letters)

# remove [:10] to view the whole song
print(merge(lyrics_nlist[:10], chords_nlist[:10]))

{t:All I Want For Christmas Is You}
{key:G}
{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}

[C♯] I don't want a lot for Christmas there is just one thing I [C♯7] need
[F♯] I don't care about the presents [F♯m] underneath the Christmas tree
[C♯] I just want you for my [C♯aug] own
[A♯m7] more than you could ever [F♯m] know
[C♯] Make my wish come [A♯7] true
[D♯madd9] All I want for [G♯7] Christmas is [C♯] you [A♯m7] [F♯] [G♯7]


Simple, isn't it? Now, let's transpose it up two semitones to E♭.

*Code*: This time, we are going to use the alternate transposing method ``transpose_simple(semitones[, use_flats=False])``*, which is useful when you just want a 'quick and dirty' transposition.

*_The use_flats flag uses flats when transposing to black keys. We would transpose to D♯ if we leave it at False._

In [ ]:
for line in chords_nlist:

    for chord in line:
        # transpose using flats on black keys
        chord.transpose_simple(2, use_flats=True)

# remove [:10] to view the whole song
print(merge(lyrics_nlist[:10], chords_nlist[:10]))

{t:All I Want For Christmas Is You}
{key:G}
{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}

[E♭] I don't want a lot for Christmas there is just one thing I [E♭7] need
[A♭] I don't care about the presents [A♭m] underneath the Christmas tree
[E♭] I just want you for my [E♭aug] own
[Cm7] more than you could ever [A♭m] know
[E♭] Make my wish come [C7] true
[Fmadd9] All I want for [B♭7] Christmas is [E♭] you [Cm7] [A♭] [B♭7]


chordparser's transpose methods account for slash chords as well, so you don't have to worry about those.

## Converting to roman numeral notation

chordparser can convert Chords to their roman numeral notation. Let's put together a new chord sheet that uses that notation:

*Code*: This is done using ``Parser.to_roman(chord, scale_key)``, where scale_key can either be the scale or key of the song. Let's iterate over the Chords nested list and try it out:

In [ ]:
chords_nlist, lyrics_nlist = get_chords_lyrics(sheet)

roman_nlist = []
for line in chords_nlist:
    new_line = []
    for chord in line:
        new_line.append(cp.to_roman(chord, scale))
    roman_nlist.append(new_line)

# remove [:10] to view the whole song
print(merge(lyrics_nlist[:10], roman_nlist[:10]))

{t:All I Want For Christmas Is You}
{key:G}
{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}

[I] I don't want a lot for Christmas there is just one thing I [I7] need
[IV] I don't care about the presents [iv] underneath the Christmas tree
[I] I just want you for my [I+] own
[vi7] more than you could ever [iv] know
[I] Make my wish come [VI7] true
[ii] All I want for [V7] Christmas is [I] you [vi7] [IV] [V7]


And that's it! This makes studying the song's chord progression much easier.

## Chord progression analysis

chordparser allows you to perform analysis on a chord relative to other chords/scales. Using this, we can extend it to analysing the chord progression of a song.

### Diatonic chords

*Definition*: chords built from the notes of a key.

*Example*: in the key of C major, the D minor chord is diatonic because its notes (D, F, A) are in the notes of C major (C, D, E, F, G, A, B).

---

Let's start off with some basic analysis: checking whether chords are diatonic. We'll flag non-diatonic chords with "!D" in the chord sheet.

*Code*: ``Parser.analyse_diatonic(chord, scale)`` returns a nested list* of the roman notation, mode and submode the chord is diatonic to, or an empty list if the chord is not diatonic. We'll use this to flag the non-diatonic chords as we iterate over the Chords nested list:

_*There might be multiple results if you use the flag ``incl_submode=True`` to check against the different types of minor keys._

In [ ]:
chords_nlist, lyrics_nlist = get_chords_lyrics(sheet)

diatonic_nlist = []
for line in chords_nlist:
    new_line = []
    for chord in line:
        results = cp.analyse_diatonic(chord, scale)

        if results:  # diatonic
            result = results[0][0]  # just show roman notation
        else:
            result = str(cp.to_roman(chord, scale)) + ", !D"

        new_line.append(result)
    diatonic_nlist.append(new_line)

# remove [:10] to view the whole song
print(merge(lyrics_nlist[:10], diatonic_nlist[:10]))

{t:All I Want For Christmas Is You}
{key:G}
{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}

[I] I don't want a lot for Christmas there is just one thing I [I7] need
[IV] I don't care about the presents [iv, !D] underneath the Christmas tree
[I] I just want you for my [I+, !D] own
[vi7] more than you could ever [iv, !D] know
[I] Make my wish come [VI7, !D] true
[ii] All I want for [V7] Christmas is [I] you [vi7] [IV] [V7]


Most of the chords are unsurprisingly diatonic, though there are a few that are not. Let's see if we can find out more about them by checking if they are borrowed chords.

### Borrowed chords (mode mixture)

*Definition*: chords taken from a key with the same root note but in a different mode, providing harmonic variety.

*Example*: an F minor chord in a song with the key of C major can be seen as a borrowed iv chord from C minor.

---

This time, we are checking for borrowed chords. We'll label them with the mode they are borrowed from.

*Code*: To check for borrowed chords, we use ``Parser.analyse_all(chord, scale)`` which checks a chord against all the possible modes. As there will be multiple results (e.g. the F minor chord in the example could also have been borrowed from C phrygian), we'll just grab the first result from ``analyse_all`` which tries to give the most likely mode. 

Let's again iterate over the Chords nested list to show the mode a Chord belongs to if it is borrowed:

In [ ]:
chords_nlist, lyrics_nlist = get_chords_lyrics(sheet)

borrowed_nlist = []
for line in chords_nlist:
    new_line = []
    for chord in line:
        results = cp.analyse_all(chord, scale)

        if results and results[0][1] != scale.key.mode:  # borrowed chord
            # show which mode it is borrowed from
            result = str(results[0][0]) + ", " + results[0][1]
        else:
            result = str(cp.to_roman(chord, scale))

        new_line.append(result)
    borrowed_nlist.append(new_line)

# remove [:10] to view the whole song
print(merge(lyrics_nlist[:10], borrowed_nlist[:10]))

{t:All I Want For Christmas Is You}
{key:G}
{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}

[I] I don't want a lot for Christmas there is just one thing I [I7] need
[IV] I don't care about the presents [iv, minor] underneath the Christmas tree
[I] I just want you for my [I+] own
[vi7] more than you could ever [iv, minor] know
[I] Make my wish come [VI7] true
[ii] All I want for [V7] Christmas is [I] you [vi7] [IV] [V7]


It turns out that the iv chords in the second and fourth lines of the verse are actually borrowed from the minor key. The second line is especially tasteful when IV changes to iv. 

Lastly, let's try to find out more about how the chords work by checking for any secondary chords.

### Secondary chords

*Definition*: chords which have the function of resolving to the next chord (which is *not* the tonic), effectively tonicizing the next chord.

*Example*: a common chord progression is the V-I authentic cadence, where the dominant resolves to the tonic. In the key of C major, a D major chord can act as a *secondary dominant* chord if it resolves to a G major chord in a V-I movement, which then resolves to C major in another V-I movement. This is denoted as V/V-V-I.

---

The most common secondary chords are secondary dominant and secondary leading tone chords, so let's check for those. We'll again label them with their secondary chord notation (e.g. V/V).

*Code*: When we check for secondary chords, we have to be keep the next chord in context. Hence, we'll check for secondary chords by going through the chord list *backwards* and compare the earlier chord against the later chord.
``Parser.analyse_secondary(prev_chord, next_chord, scale)`` will help us to identify the roman numeral notation of any secondary dominant or leading tone chords*.

You should know the drill by now - time to iterate over the Chords nested list:

_*Use the flag ``limit=False`` to check for all types of secondary chords._

In [ ]:
chords_nlist, lyrics_nlist = get_chords_lyrics(sheet)

reverse_nlist = chords_nlist[::-1]
secondary_nlist = []
next_chord = cp.create_diatonic(scale, 1)  # use tonic as starting basis

for line in reverse_nlist:
    reverse_line = line[::-1]
    new_line = []

    for prev_chord in reverse_line:
        roman = cp.to_roman(prev_chord, scale)
        result = cp.analyse_secondary(
            prev_chord, next_chord, scale, incl_submodes=True  # account for harmonic minor
        )
        if result:
            roman = str(roman) + " (" + result + ")"
        new_line.append(roman)
        next_chord = prev_chord
        
    secondary_nlist.append(new_line[::-1])
secondary_nlist = secondary_nlist[::-1]

# remove [:10] to view the whole song
print(merge(lyrics_nlist[:10], secondary_nlist[:10]))

{t:All I Want For Christmas Is You}
{key:G}
{c: Source - https://ozbcoz.com/Songs/song.php?ID=4317}

[I] I don't want a lot for Christmas there is just one thing I [I7 (V7/IV)] need
[IV] I don't care about the presents [iv] underneath the Christmas tree
[I] I just want you for my [I+] own
[vi7] more than you could ever [iv] know
[I] Make my wish come [VI7 (V7/ii)] true
[ii] All I want for [V7] Christmas is [I] you [vi7] [IV] [V7]


Here, we see two instances of secondary dominant chords. The I7 in "there is just one thing I need" could have been mistaken as just a tonic chord with a dominant seventh, but it actually has the function of leading to the IV chord. *(This is usually confirmed by the I7 being a dominant chord to emphasize its dominant function, instead of being a I chord.)*

It also turns out that the non-diatonic VI7 chord in "Make my wish come true" acts as a secondary dominant to lead to the ii chord, making it V7/ii.

---
And there you have it - with the help of chordparser, the chord progression of this song is demystified a little. Out of the 4 non-diatonic chords, we've started to see why 3 of them were placed in the song and even had a bonus tidbit on the I7 chord. Why not make a copy of this notebook and try it out for yourself?

*To end on a satisfying note: the last non-diatonic chord is actually using a chromatic nonchord tone. Notice how the chord before is I (I, III, V) and the chord after is iv (iv, I, III). The augmented fifth acts as a step between V and iv.*

# Ending notes

This working example of understanding chord progressions is one possibility of using chordparser. If you have any problems or suggestions on how it can be improved or used, I would greatly appreciate it if you could create an issue on the [Github Issue Tracker](https://github.com/titus-ong/chordparser/issues).